### It's time to train the neural network






Train the shit bro

In [3]:
import tensorflow as tf
import numpy as np
import chess


In [4]:

# Load the training data from saved numpy arrays
# positions = np.load('SL_boards2e+04.npy')
# best_moves = np.load('SL_best_moves2e+04.npy')
# best_moves_probs = np.load('SL_best_moves_probs2e+04.npy')
# outcomes = np.load('outcomes2e+04.npy')



positions = np.load('final_endgame_pos.npy')
# best_moves = np.load('SL_best_moves2e+04.npy')
best_moves_probs = np.load('final_endgame_move_probs.npy')
outcomes = np.load('final_endgame_evaluations.npy')

In [5]:

# Define a function to create the neural network model
def create_model():
    # Define the input layer
    inp = tf.keras.layers.Input((8, 8, 25))

    # Add the first convolutional layer
    l1 = tf.keras.layers.Conv2D(filters=256, kernel_size=3, padding='same', data_format='channels_last')(inp)
    l1 = tf.keras.layers.BatchNormalization()(l1)
    l1 = tf.keras.layers.Activation('relu')(l1)

    # Add 5 residual blocks (each consisting of two convolutional layers with skip connections)
    n_layers = 5
    for i in range(n_layers):
        temp = l1
        l1 = tf.keras.layers.Conv2D(filters=256, kernel_size=3, padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-5), data_format='channels_last')(l1)
        l1 = tf.keras.layers.BatchNormalization()(l1)
        l1 = tf.keras.layers.Activation('relu')(l1)
        l1 = tf.keras.layers.Conv2D(filters=256, kernel_size=3, padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-5), data_format='channels_last')(l1)
        l1 = tf.keras.layers.BatchNormalization()(l1)
        l1 = tf.keras.layers.Add()([l1, temp])
        l1 = tf.keras.layers.Activation('relu')(l1)

    # Add the policy head
    l2 = tf.keras.layers.Conv2D(filters=2, kernel_size=1, padding='same', data_format='channels_last')(l1)
    l2 = tf.keras.layers.Flatten()(l2)
    policyOut = tf.keras.layers.Dense(1968, name='policyHead', activation='softmax')(l2)

    # Add the value head
    l3 = tf.keras.layers.Conv2D(filters=1, kernel_size=1, padding='same', data_format='channels_last')(l1)
    l4 = tf.keras.layers.BatchNormalization()(l3)
    l5 = tf.keras.layers.Activation('relu')(l4)
    l5 = tf.keras.layers.Flatten()(l5)
    l6 = tf.keras.layers.Dense(256, activation='relu')(l5)
    valueOut = tf.keras.layers.Dense(1, activation='tanh', name='valueHead')(l6)

    # Define the loss function and compile the model
    bce = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
    model = tf.keras.Model(inp, [policyOut, valueOut])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4, beta_1=0.9, beta_2=0.999, amsgrad=True),
                  loss={'valueHead': 'mean_squared_error', 'policyHead': bce})
    
    return model


In [6]:

# Create the model
model = create_model()

# Save the initial (randomly initialized) model
model.save('random_model.h5')

# Define the inputs and outputs for training
inputData = positions
policyOutcomes = best_moves_probs
valueOutcomes = outcomes

# Define early stopping and learning rate reduction callbacks
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, patience=15, mode='min', verbose=1)
reduce_lr1 = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_valueHead_loss', factor=0.95, patience=1, min_lr=1e-7)
reduce_lr2 = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_policyHead_loss', factor=0.95, patience=1, min_lr=1e-7)

# Define a checkpoint callback to save the best model during training
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='./checkpoints', save_best_only=True, save_weights_only=True)

# Train the model using the defined inputs and outputs, with the defined callbacks
model.fit(inputData, [policyOutcomes, valueOutcomes], validation_split=0.2, epochs=40, batch_size=100,
          validation_steps=100, steps_per_epoch=1500, callbacks=[es, reduce_lr1, reduce_lr2, checkpoint_callback])

# Save the trained model
model.save('supervisedaregtaeg_model__.h5')

Epoch 1/40
1500/1500 [==============================] - 811s 539ms/step - loss: 4.4564 - policyHead_loss: 4.4223 - valueHead_loss: 0.0081 - val_loss: 4.0450 - val_policyHead_loss: 4.0107 - val_valueHead_loss: 0.0079 - lr: 1.0000e-04
Epoch 2/40
1500/1500 [==============================] - 751s 501ms/step - loss: 4.3766 - policyHead_loss: 4.3421 - valueHead_loss: 0.0073 - val_loss: 5.2290 - val_policyHead_loss: 5.1936 - val_valueHead_loss: 0.0076 - lr: 1.0000e-04
Epoch 3/40
1500/1500 [==============================] - 742s 495ms/step - loss: 5.9956 - policyHead_loss: 5.9600 - valueHead_loss: 0.0071 - val_loss: 6.9005 - val_policyHead_loss: 6.8640 - val_valueHead_loss: 0.0076 - lr: 9.0250e-05
Epoch 4/40
1500/1500 [==============================] - 742s 495ms/step - loss: 8.3594 - policyHead_loss: 8.3231 - valueHead_loss: 0.0069 - val_loss: 9.7647 - val_policyHead_loss: 9.7274 - val_valueHead_loss: 0.0074 - lr: 8.5737e-05
Epoch 5/40
 625/1500 [===========>..................] - ETA: 7:06 - 

KeyboardInterrupt: 

In [4]:

# Define a function to create the neural network model
def create_policy_model():
    # Define the input layer
    inp = tf.keras.layers.Input((8, 8, 25))

    # Add the first convolutional layer
    l1 = tf.keras.layers.Conv2D(filters=256, kernel_size=3, padding='same', data_format='channels_last')(inp)
    l1 = tf.keras.layers.BatchNormalization()(l1)
    l1 = tf.keras.layers.Activation('relu')(l1)

    # Add 5 residual blocks (each consisting of two convolutional layers with skip connections)
    n_layers = 5
    for i in range(n_layers):
        temp = l1
        l1 = tf.keras.layers.Conv2D(filters=256, kernel_size=3, padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-5), data_format='channels_last')(l1)
        l1 = tf.keras.layers.BatchNormalization()(l1)
        l1 = tf.keras.layers.Activation('relu')(l1)
        l1 = tf.keras.layers.Conv2D(filters=256, kernel_size=3, padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-5), data_format='channels_last')(l1)
        l1 = tf.keras.layers.BatchNormalization()(l1)
        l1 = tf.keras.layers.Add()([l1, temp])
        l1 = tf.keras.layers.Activation('relu')(l1)

    # Add the policy head
    l2 = tf.keras.layers.Conv2D(filters=2, kernel_size=1, padding='same', data_format='channels_last')(l1)
    l2 = tf.keras.layers.Flatten()(l2)
    policyOut = tf.keras.layers.Dense(1968, name='policyHead', activation='softmax')(l2)

    # Define the loss function and compile the model
    bce = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
    model = tf.keras.Model(inp, policyOut)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4, beta_1=0.9, beta_2=0.999, amsgrad=True),
                  loss={ 'policyHead': bce})
    
    return model


In [5]:
model=create_policy_model()

# Save the initial (randomly initialized) model
model.save('random_policy_model.h5')

# Define the inputs and outputs for training


inputData = np.load('x_puzzles1m.npy')
policyOutcomes = np.load('y_puzzles1m.npy')


# Define early stopping and learning rate reduction callbacks
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, patience=15, mode='min', verbose=1)
reduce_lr1 = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_valueHead_loss', factor=0.95, patience=2, min_lr=1e-7)
reduce_lr2 = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_policyHead_loss', factor=0.95, patience=2, min_lr=1e-7)

# Define a checkpoint callback to save the best model during training
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='./checkpoints', save_best_only=True, save_weights_only=True)

# Train the model using the defined inputs and outputs, with the defined callbacks
model.fit(inputData, policyOutcomes, validation_split=0.2, epochs=40, batch_size=100,
          validation_steps=100, steps_per_epoch=1500, callbacks=[es, reduce_lr1, reduce_lr2, checkpoint_callback])

# Save the trained model
model.save('SUPERVISED_policy_model.h5')

Epoch 1/40
1500/1500 [==============================] - 822s 546ms/step - loss: 6.0916 - val_loss: 4.7997 - lr: 1.0000e-04
Epoch 2/40
1500/1500 [==============================] - 767s 511ms/step - loss: 3.9450 - val_loss: 3.3325 - lr: 1.0000e-04
Epoch 3/40
1500/1500 [==============================] - 774s 516ms/step - loss: 3.0342 - val_loss: 2.7560 - lr: 1.0000e-04
Epoch 4/40
1500/1500 [==============================] - 800s 533ms/step - loss: 2.6233 - val_loss: 2.4416 - lr: 1.0000e-04
Epoch 5/40
1500/1500 [==============================] - 792s 528ms/step - loss: 2.3688 - val_loss: 2.2524 - lr: 1.0000e-04
Epoch 6/40
1500/1500 [==============================] - 785s 523ms/step - loss: 2.2062 - val_loss: 2.0952 - lr: 1.0000e-04
Epoch 7/40
1500/1500 [==============================] - 788s 525ms/step - loss: 2.0807 - val_loss: 2.0197 - lr: 1.0000e-04
Epoch 8/40
1500/1500 [==============================] - 785s 523ms/step - loss: 1.9741 - val_loss: 1.9031 - lr: 1.0000e-04
Epoch 9/40
1500/

1500/1500 [==============================] - 783s 522ms/step - loss: 1.5185 - val_loss: 1.5195 - lr: 1.0000e-04
Epoch 18/40
1500/1500 [==============================] - 781s 521ms/step - loss: 1.4948 - val_loss: 1.5013 - lr: 1.0000e-04
Epoch 19/40
1500/1500 [==============================] - 782s 521ms/step - loss: 1.4859 - val_loss: 1.4811 - lr: 1.0000e-04
Epoch 20/40
1500/1500 [==============================] - 781s 521ms/step - loss: 1.4102 - val_loss: 1.4692 - lr: 1.0000e-04
Epoch 21/40
1500/1500 [==============================] - 783s 522ms/step - loss: 1.3055 - val_loss: 1.4875 - lr: 1.0000e-04
Epoch 22/40
1500/1500 [==============================] - 782s 521ms/step - loss: 1.3166 - val_loss: 1.4542 - lr: 1.0000e-04
Epoch 23/40
1500/1500 [==============================] - 781s 521ms/step - loss: 1.3144 - val_loss: 1.4550 - lr: 1.0000e-04
Epoch 24/40
1500/1500 [==============================] - 777s 518ms/step - loss: 1.3151 - val_loss: 1.4327 - lr: 1.0000e-04
Epoch 25/40
1500/150

1500/1500 [==============================] - 732s 488ms/step - loss: 1.1535 - val_loss: 1.3339 - lr: 1.0000e-04
Epoch 35/40
1500/1500 [==============================] - 731s 487ms/step - loss: 1.1541 - val_loss: 1.3215 - lr: 1.0000e-04
Epoch 36/40
1500/1500 [==============================] - 742s 495ms/step - loss: 1.1550 - val_loss: 1.3182 - lr: 1.0000e-04
Epoch 37/40
1500/1500 [==============================] - 739s 493ms/step - loss: 1.1506 - val_loss: 1.3110 - lr: 1.0000e-04
Epoch 38/40
1500/1500 [==============================] - 749s 500ms/step - loss: 1.1615 - val_loss: 1.3183 - lr: 1.0000e-04
Epoch 39/40
1500/1500 [==============================] - 769s 512ms/step - loss: 1.1517 - val_loss: 1.3161 - lr: 1.0000e-04
Epoch 40/40
1500/1500 [==============================] - 742s 495ms/step - loss: 1.0253 - val_loss: 1.3347 - lr: 1.0000e-04
